-sandbox

<div style="text-align: center; line-height: 0; padding-top: 9px;">
  <img src="https://databricks.com/wp-content/uploads/2018/03/db-academy-rgb-1200px.png" alt="Databricks Learning" style="width: 600px">
</div>

<i18n value="b9ee68dd-a00f-45a6-9c7a-9c59a41fef6a"/>


# SQL for Delta Live Tables

In the last lesson, we walked through the process of scheduling this notebook as a Delta Live Table (DLT) pipeline. Now we'll explore the contents of this notebook to better understand the syntax used by Delta Live Tables.

This notebook uses SQL to declare Delta Live Tables that together implement a simple multi-hop architecture based on a Databricks-provided example dataset loaded by default into Databricks workspaces.

At its simplest, you can think of DLT SQL as a slight modification to traditional CTAS statements. DLT tables and views will always be preceded by the **`LIVE`** keyword.

## Learning Objectives
By the end of this lesson, you should be able to:
* Define tables and views with Delta Live Tables
* Use SQL to incrementally ingest raw data with Auto Loader
* Perform incremental reads on Delta tables with SQL
* Update code and redeploy a pipeline

<i18n value="1cc1fe77-a235-40dd-beb5-bacc94425b96"/>


## Declare Bronze Layer Tables

Below we declare two tables implementing the bronze layer. This represents data in its rawest form, but captured in a format that can be retained indefinitely and queried with the performance and benefits that Delta Lake has to offer.

<i18n value="458afeb2-4472-4ed2-a4c7-6dea2338c3f2"/>


### sales_orders_raw

**`sales_orders_raw`** ingests JSON data incrementally from our **retail-org/sales_orders** dataset.

Incremental processing via <a herf="https://docs.databricks.com/spark/latest/structured-streaming/auto-loader.html" target="_blank">Auto Loader</a> (which uses the same processing model as Structured Streaming), requires the addition of the **`STREAMING`** keyword in the declaration as seen below. The **`cloud_files()`** method enables Auto Loader to be used natively with SQL. This method takes the following positional parameters:
* The source location, as mentioned above
* The source data format, which is JSON in this case
* An arbitrarily sized array of optional reader options. In this case, we set **`cloudFiles.inferColumnTypes`** to **`true`**

The following declaration also demonstrates the declaration of additional table metadata (a comment and properties in this case) that would be visible to anyone exploring the data catalog.

In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE sales_orders_raw
COMMENT "The raw sales orders, ingested from retail-org/sales_orders."
AS SELECT * FROM cloud_files("${datasets_path}/retail-org/sales_orders", "json", map("cloudFiles.inferColumnTypes", "true"))

message
"This Delta Live Tables query is syntactically valid, but you must create a pipeline in order to define and populate your table."


<i18n value="57accbe1-e96d-44ca-8937-8849634d1da2"/>


### customers

**`customers`** presents CSV customer data found in **retail-org/customers**.

This table will soon be used in a join operation to look up customer data based on sales records.

In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE customers
COMMENT "The customers buying finished products, ingested from retail-org/customers."
AS SELECT * FROM cloud_files("${datasets_path}/retail-org/customers/", "csv");

message
"This Delta Live Tables query is syntactically valid, but you must create a pipeline in order to define and populate your table."


<i18n value="09bc5c4d-1408-47f8-8102-ddaf0a96a6c0"/>



## Declare Silver Layer Tables

Now we declare tables implementing the silver layer. This layer represents a refined copy of data from the bronze layer, with the intention of optimizing downstream applications. At this level we apply operations like data cleansing and enrichment.

<i18n value="eb20ea4d-1115-4536-adac-7bbd85491726"/>


### sales_orders_cleaned

Here we declare our first silver table, which enriches the sales transaction data with customer information in addition to implementing quality control by rejecting records with a null order number.

This declaration introduces a number of new concepts.

#### Quality Control

The **`CONSTRAINT`** keyword introduces quality control. Similar in function to a traditional **`WHERE`** clause, **`CONSTRAINT`** integrates with DLT, enabling it to collect metrics on constraint violations. Constraints provide an optional **`ON VIOLATION`** clause, specifying an action to take on records that violate the constraint. The three modes currently supported by DLT include:

| **`ON VIOLATION`** | Behavior |
| --- | --- |
| **`FAIL UPDATE`** | Pipeline failure when constraint is violated |
| **`DROP ROW`** | Discard records that violate constraints |
| Omitted | Records violating constraints will be included (but violations will be reported in metrics) |

#### References to DLT Tables and Views
References to other DLT tables and views will always include the **`live.`** prefix. A target database name will automatically be substituted at runtime, allowing for easily migration of pipelines between DEV/QA/PROD environments.

#### References to Streaming Tables

References to streaming DLT tables use the **`STREAM()`**, supplying the table name as an argument.

In [0]:
%sql
CREATE OR REFRESH STREAMING LIVE TABLE sales_orders_cleaned(
  CONSTRAINT valid_order_number EXPECT (order_number IS NOT NULL) ON VIOLATION DROP ROW
)
COMMENT "The cleaned sales orders with valid order_number(s)."
AS
  SELECT f.customer_id, f.customer_name, f.number_of_line_items, 
         timestamp(from_unixtime((cast(f.order_datetime as long)))) as order_datetime, 
         date(from_unixtime((cast(f.order_datetime as long)))) as order_date, 
         f.order_number, f.ordered_products, c.state, c.city, c.lon, c.lat, c.units_purchased, c.loyalty_segment
  FROM STREAM(LIVE.sales_orders_raw) f
  LEFT JOIN LIVE.customers c
    ON c.customer_id = f.customer_id
    AND c.customer_name = f.customer_name

message
"This Delta Live Tables query is syntactically valid, but you must create a pipeline in order to define and populate your table."


<i18n value="ae062501-5a39-4183-976a-53662619d516"/>


## Declare Gold Table

At the most refined level of the architecture, we declare a table delivering an aggregation with business value, in this case a collection of sales order data based in a specific region. In aggregating, the report generates counts and totals of orders by date and customer.

In [0]:
%sql
CREATE OR REFRESH LIVE TABLE sales_order_in_la
COMMENT "Sales orders in LA."
AS
  SELECT city, order_date, customer_id, customer_name, ordered_products_explode.curr, 
         sum(ordered_products_explode.price) as sales, 
         sum(ordered_products_explode.qty) as quantity, 
         count(ordered_products_explode.id) as product_count
  FROM (SELECT city, order_date, customer_id, customer_name, explode(ordered_products) as ordered_products_explode
        FROM LIVE.sales_orders_cleaned 
        WHERE city = 'Los Angeles')
  GROUP BY order_date, city, customer_id, customer_name, ordered_products_explode.curr

message
"This Delta Live Tables query is syntactically valid, but you must create a pipeline in order to define and populate your table."


<i18n value="c3efd15d-6587-4cd4-9758-37643dcfd694"/>


## Explore Results

Explore the DAG (Directed Acyclic Graph) representing the entities involved in the pipeline and the relationships between them. Click on each to view a summary, which includes:
* Run status
* Metadata summary
* Schema
* Data quality metrics

Refer to this <a href="$./DE 8.3 - Pipeline Results" target="_blank">companion notebook</a> to inspect tables and logs.

<i18n value="77c262ce-1b24-4ca2-a931-e481927d1739"/>


## Update Pipeline

Uncomment the following cell to declare another gold table. Similar to the previous gold table declaration, this filters for the **`city`** of Chicago. 

Re-run your pipeline to examine the updated results. 

Does it run as expected? 

Can you identify any issues?

In [0]:
%sql
-- ANSWER
CREATE OR REFRESH LIVE TABLE sales_order_in_chicago
COMMENT "Sales orders in Chicago."
AS
  SELECT city, order_date, customer_id, customer_name, ordered_products_explode.curr, 
         sum(ordered_products_explode.price) as sales, 
         sum(ordered_products_explode.qty) as quantity, 
         count(ordered_products_explode.id) as product_count
  FROM (SELECT city, order_date, customer_id, customer_name, explode(ordered_products) as ordered_products_explode
        FROM LIVE.sales_orders_cleaned 
        WHERE city = 'Chicago')
  GROUP BY order_date, city, customer_id, customer_name, ordered_products_explode.curr

message
"This Delta Live Tables query is syntactically valid, but you must create a pipeline in order to define and populate your table."


-sandbox
&copy; 2022 Databricks, Inc. All rights reserved.<br/>
Apache, Apache Spark, Spark and the Spark logo are trademarks of the <a href="https://www.apache.org/">Apache Software Foundation</a>.<br/>
<br/>
<a href="https://databricks.com/privacy-policy">Privacy Policy</a> | <a href="https://databricks.com/terms-of-use">Terms of Use</a> | <a href="https://help.databricks.com/">Support</a>